(sec:resp_charges)=
# RESP charges

The restrained electrostatic potential (RESP) charge model represents an improvement to the Merz–Kollman scheme as the [ESP figure-of-merit](sec:esp_charges) is rather insensitive to variations in charges of atoms buried inside the molecule.

```{figure} ../img/chi_square.svg
---
name: chi_square
width: 600px
align: center
---
Dependence of figure-of-merit, $\chi^2_\mathrm{esp}$, with respect to variations in atomic charges. Four separate atoms are here considered.
```

To avoid unphysically high magnitudes of the charges of interior atoms, a hyperbolic penalty function is added

\begin{equation*}
\chi_{\mathrm{rstr}}^2 = \alpha \sum_I \bigl((q_I^2+\beta^2)^{1/2}-\beta\bigl)
\end{equation*}

so that the diagonal matrix elements of the $A$-matrix in Merz–Kollman scheme become equal to

\begin{equation*}
A_{JJ} = 
\frac{1}{4\pi\varepsilon_0}
\sum_{a} \frac{1}{r_{aJ}^2} + \alpha \, (q_J^2+\beta^2)^{-1/2}
\end{equation*}

with a dependency on the partial charge. Consequently, RESP charges are obtained by solving the matrix equation iteratively until the charges and Lagrange multipliers become self-consistent. In addition to that, the RESP charge model allows for the introduction of constraints on charges of equivalent atoms due to symmetry operations or bond rotations.

In [2]:
import py3Dmol as p3d

methanol_xyz = """
  H      1.2001      0.0363      0.8431
  C      0.7031      0.0083     -0.1305
  H      0.9877      0.8943     -0.7114
  H      1.0155     -0.8918     -0.6742
  O     -0.6582     -0.0067      0.1730
  H     -1.1326     -0.0311     -0.6482
"""

v = p3d.view(width=400, height=200)

v.addModel("6\n" + methanol_xyz, "xyz")
v.setStyle({'stick':{}})
v.zoomTo()
v.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [3]:
import veloxchem as vlx

molecule = vlx.Molecule.read_str(methanol_xyz, units = 'angstrom')
basis = vlx.MolecularBasis.read(molecule, "def2-SVP")

print(basis.get_string(molecule))

* Warning * Environment variable OMP_NUM_THREADS not set.
* Warning * Setting OMP_NUM_THREADS to 8.
Molecular Basis (Atomic Basis)

Basis: DEF2-SVP                                       

  Atom Contracted GTOs          Primitive GTOs           

  O   (3S,2P,1D)               (7S,4P,1D)               
  H   (2S,1P)                  (4S,1P)                  
  C   (3S,2P,1D)               (7S,4P,1D)               

Contracted Basis Functions : 48                       
Primitive Basis Functions  : 76                       




A calculation of RESP charges is performed by invoking the `RespChargesDriver` driver class.

In [15]:
resp_drv = vlx.RespChargesDriver()

resp_drv.update_settings({
    'equal_charges': '1 = 3, 1 = 4'
})

resp_charges = resp_drv.compute(molecule, basis, 'resp')

                                  *** Warning: Recommended basis set 6-31G* is not used!                                  
                                                                                                                          
                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Hartree-Fock                                         
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                

                                                                                                                          
                                                RESP Charges Driver Setup                                                 
                                                                                                                          
                                         Number of Conformers         :  1                                                
                                         Number of Layers             :  4                                                
                                         Points per Square Angstrom   :  1.0                                              
                                         Total Number of Grid Points  :  420                                              
                                                                                                                          
                

In [16]:
print("Atom   Charge (a.u.)")
print("-"*20)

for label, charge in zip(molecule.get_labels(), resp_charges):
    print(f"{label :s} {charge : 18.6f}")

print("-"*20)
print(f"Total: {resp_charges.sum() : 13.6f}")

Atom   Charge (a.u.)
--------------------
H           0.022789
C           0.159653
H           0.022789
H           0.022789
O          -0.633671
H           0.405651
--------------------
Total:      0.000000
